#### This assignment may be worked individually or in pairs. Enter your name/s here:
    

In [64]:
#Ritu Gupta 

# Assignment 2: Decision Trees

In this assignment we'll implement the Decision Tree algorithm to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains features from the Messidor image set to predict whether an image contains signs of diabetic retinopathy or not. This dataset has `1150` records and `20` attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

Attribute Information:

0) The binary result of quality assessment. 0 = bad quality 1 = sufficient quality.

1) The binary result of pre-screening, where 1 indicates severe retinal abnormality and 0 its lack. 

2-7) The results of MA detection. Each feature value stand for the number of MAs found at the confidence levels alpha = 0.5, . . . , 1, respectively. 

8-15) contain the same information as 2-7) for exudates. However, as exudates are represented by a set of points rather than the number of pixels constructing the lesions, these features are normalized by dividing the 
number of lesions with the diameter of the ROI to compensate different image sizes. 

16) The euclidean distance of the center of the macula and the center of the optic disc to provide important information regarding the patient's condition. This feature is also normalized with the diameter of the ROI.

17) The diameter of the optic disc. 

18) The binary result of the AM/FM-based classification.

19) Class label. 1 = contains signs of Diabetic Retinopathy, 0 = no signs of Diabetic Retinopathy.

#### Implementation: 
The function prototypes are given to you, please don't change those. You can add additional helper functions if needed. 

*Suggestion:* The dataset is substantially big, for the purpose of easy debugging, work with a subset of the data and test your decision tree implementation on that.

#### Notes:
Parts of this assignment will be **autograded** so a couple of caveats :-
- Entropy is calculated using log with base 2, `math.log2(x)`.
- For continuous features ensure that the threshold value lies exactly between 2 values. For example, if for feature 2 the best split occurs between 10 and 15 then the threshold value will be set as 12.5. For binary features [0/1] the threshold value will be 0.5.
- All values < `thresh_val` go to the left child and all values >= `thresh_val` go to the right child.

In [65]:
# Standard Headers
# You are welcome to add additional headers if you wish
# EXCEPT for scikit-learn... You may NOT use scikit-learn for this assignment!
import pandas as pd
import numpy as np
from math import log2
from math import floor
from collections import Counter

In [66]:
class DataPoint:
    def __str__(self):
        return "< " + str(self.label) + ": " + str(self.features) + " >"
    
    def __init__(self, label, features):
        self.label = label # the classification label of this data point
        self.features = features # a list of feature values for this data point

Q1. Read data from a CSV file. Put it into a list of `DataPoints`.

In [67]:
def get_data(filename):
    data = []
#   your code goes here
    df = pd.read_csv(filename, header=None).to_numpy()
    for dp in df:
        #class label and attributes
        data.append(DataPoint(dp[19], dp[0:18]))

    return data

In [68]:
class TreeNode:
    is_leaf = True          # boolean variable to check if the node is a leaf
    feature_idx = None      # index that identifies the feature
    thresh_val = None       # threshold value that splits the node
    prediction = None       # prediction class (only valid for leaf nodes)
    left_child = None       # left TreeNode (all values < thresh_val)
    right_child = None      # right TreeNode (all values >= thresh_val)
    
    def printTree(self, level=0):    # for debugging purposes
        if self.is_leaf:
            print ('-'*level + 'Leaf Node:      predicts ' + str(self.prediction))
        else:
            print ('-'*level + 'Internal Node:  splits on feature ' 
                   + str(self.feature_idx) + ' with threshold ' + str(self.thresh_val))
            self.left_child.printTree(level+1)
            self.right_child.printTree(level+1)

Q2. Implement the function `make_prediction` that takes the decision tree root and a `DataPoint` instance and returns the prediction label.

In [69]:
def make_prediction(tree_root, data_point):
#     your code goes here
    node = tree_root
    #if there are still more steps to be taken
    while node.is_leaf != True:
        #if the value is less than, goes to the left child
        if data_point.features[node.feature_idx] < node.thresh_val:
            node = node.left_child
        else:
            node = node.right_child
    #prediction class
    return node.prediction

Q3. Implement the function `split_dataset` given an input data set, a `feature_idx` and the `threshold` for the feature. `left_split` will have all values < `threshold` and `right_split` will have all values >= `threshold`.

In [70]:
def split_dataset(data, feature_idx, threshold):
    left_split = []
    right_split = []
#     your code goes here
    for dp in data:
        if dp.features[feature_idx] >= threshold:
            right_split.append(dp)
        else:
            left_split.append(dp)
    return (left_split, right_split)

Q4. Implement the function `calc_entropy` to return the entropy of the input dataset.

In [71]:
def calc_entropy(data):
    entropy = 0.0
#     your code goes here
    num_instances = len(data)  #all instances in ds

    #entropy 0 if ds empty 
    if num_instances == 0:
        return entropy

    #frequency of each class label 
    label_counts = Counter(point.label for point in data)

    # Calculate entropy
    for label_count in label_counts.values():
        #prob of each class label
        pi = label_count / num_instances
        # Update entropy
        entropy -= pi * log2(pi)
    return entropy

Q5. Implement the function `calc_best_threshold` which returns the best information gain and the corresponding threshold value for one feature at `feature_idx`.

In [72]:
def calc_best_threshold(data, feature_idx):
    best_info_gain = 0.0
    best_thresh = None
#     your code goes here
     #sort based on the feature @ feature_idx
    sorted_data = sorted(data, key=lambda x: x.features[feature_idx])

    # iteratre through to find best threshold
    for i in range(1, len(sorted_data)):
        # calculate midpoint 
        thresh = (sorted_data[i - 1].features[feature_idx] + sorted_data[i].features[feature_idx]) / 2
        
        # split based on the current threshold
        left_split, right_split = split_dataset(sorted_data, feature_idx, thresh)
        
        # entropy for left and right splits
        entropy_left = calc_entropy(left_split)
        entropy_right = calc_entropy(right_split)
        
        #find info gain
        info_gain = calc_entropy(sorted_data) - (len(left_split) / len(sorted_data) * entropy_left + len(right_split) / len(sorted_data) * entropy_right)
        
        #update info gain and threshold
        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_thresh = thresh
    
    return (best_info_gain, best_thresh)

Q6. Implement the function `identify_best_split` which returns the best feature to split on for an input dataset, and also returns the corresponding threshold value.

In [73]:
def identify_best_split(data):
    if len(data) < 2:
        return (None, None)
    best_feature = None
    best_thresh = None
#     your code goes here
    num_features = len(data[0].features)  # num features in ds
    best_info_gain = 0.0

    #going through each feature
    for feature_idx in range(num_features):
        # calc best threshold for curr feature
        feature_info_gain, feature_thresh = calc_best_threshold(data, feature_idx)
        
        # Update if curr feature's info gain is higher
        if feature_info_gain > best_info_gain:
            best_info_gain = feature_info_gain
            best_feature = feature_idx
            best_thresh = feature_thresh
    
    return (best_feature, best_thresh)

Q7. Implement the function `create_leaf_node` which returns a `TreeNode` with `is_leaf=True` and `prediction` set to whichever classification occurs most in the dataset at this node. If there is a tie, choose classification label 1 (has disease). 

In [74]:
def create_leaf_node(data):
#     your code goes here
    #counting how many times each class occurs 
    class_0 = 0
    class_1 = 0

    # Count occurrences of each class label in the dataset
    for dp in data:
        if dp.label == 0:
            class_0 += 1
        else:
            class_1 += 1

    #finding which one occurred more 
    if class_0 >= class_1:
        prediction = 0
    else:
        prediction = 1

    #making node for the leaf
    leaf_node = TreeNode()
    leaf_node.is_leaf = True
    leaf_node.prediction = prediction
    
    return leaf_node

Q8. Implement the `create_decision_tree` function. `max_levels` denotes the maximum height of the tree (for example if `max_levels = 1` then the decision tree will only contain the leaf node at the root). [Hint: this is where the recursion happens.]

In [75]:
def create_decision_tree(data, max_levels):
#     your code goes here
    if max_levels == 1:
        return create_leaf_node(data)
    
    #finding what's best to split on
    best_feature, best_thresh = identify_best_split(data)
    
    #leaf node if no best feature 
    if best_feature == None:
        return create_leaf_node(data)
    
    #splitting based on feature and threshold
    left_split, right_split = split_dataset(data, best_feature, best_thresh)
    
    node = TreeNode()
    node.is_leaf = False
    node.thresh_val = best_thresh
    node.feature_idx = best_feature
    
    #building the subtrees
    node.right_child = create_decision_tree(right_split, max_levels - 1)
    node.left_child = create_decision_tree(left_split, max_levels - 1)    
    
    return node

Q9. Given a test set, the function `calc_accuracy` returns the accuracy of the classifier. You'll use the `make_prediction` function for this.

In [76]:
def calc_accuracy(tree_root, data):
#     your code goes here
    accurate = 0
    
    for data_point in data:
        if data_point.label == make_prediction(tree_root, data_point):
            accurate += 1
            
    #return the accuracy
    accuracy = accurate / len(data)
    return accuracy

Q10. Keeping the `max_levels` parameter as 10, use 5-fold cross validation to measure the accuracy of the model. Print the accuracy of the model.

In [77]:
# edit the code here - this is just a sample to get you started
import time

d = get_data("messidor_features.txt")
full_accuracy = 0.0
k = 5 # number of folds

# partition data into train_set and test_set
for i in range(0, k):
    # Partition data into train_set and test_set
    test_start = floor((i / k) * len(d))
    test_end = floor(((i + 1) / k) * len(d))
    test_set = d[test_start: test_end]
    train_set = d[0: test_start] + d[test_end: len(d)]

    print ('Training set size:', len(train_set))
    print ('Test set size    :', len(test_set))

    # the timer is just for fun! you will NOT be graded on runtime
    start = time.time()

    # create the decision tree
    tree = create_decision_tree(train_set, 10)

    end = time.time()
    print ('Time taken:', end - start)

    # calculate the accuracy of the tree
    accuracy = calc_accuracy(tree, test_set)
    full_accuracy = full_accuracy + accuracy
    print ('Accuracy on the test set:', str(accuracy * 100.0))


print('Avg accuracy of model: ', str((full_accuracy / k) * 100))
#t.printTree()

Training set size: 920
Test set size    : 230
Time taken: 11.533660888671875
Accuracy on the test set: 64.78260869565217
Training set size: 920
Test set size    : 230
Time taken: 10.789817094802856
Accuracy on the test set: 62.17391304347826
Training set size: 920
Test set size    : 230
Time taken: 10.628371000289917
Accuracy on the test set: 66.95652173913044
Training set size: 920
Test set size    : 230
Time taken: 10.66664171218872
Accuracy on the test set: 63.91304347826087
Training set size: 920
Test set size    : 230
Time taken: 10.637266874313354
Accuracy on the test set: 63.91304347826087
Avg accuracy of model:  64.34782608695653
